In [4]:
import numpy as np

def get_distance(self, height_feature, x_ori, is_equal=True, neg=False, threshold=0):
    plane_feature = np.ones_like(height_feature, dtype=np.float32)
    
    if x_ori:  # entlang X-Richtung
        count = np.ones((self.bin_size_x, self.bin_size_y), dtype=np.float32)
        if neg:
            count[0, :] = 0  # erste Zeile darf nicht mitzählen
            for x in range(1, self.bin_size_x):
                comp = abs(height_feature[x, :] - height_feature[x-1, :]) <= threshold
                count[x, :] = np.where(comp, count[x-1, :] + 1, 0)
        else:
            count[-1, :] = 1
            for x in range(self.bin_size_x-2, -1, -1):
                if is_equal:
                    comp = abs(height_feature[x, :] - height_feature[x+1, :]) <= threshold
                else:
                    comp = height_feature[x, :] >= height_feature[x+1, :]
                count[x, :] = np.where(comp, count[x+1, :] + 1, 1)
        plane_feature = count

    else:  # entlang Y-Richtung
        count = np.ones((self.bin_size_x, self.bin_size_y), dtype=np.float32)
        if neg:
            count[:, 0] = 0
            for y in range(1, self.bin_size_y):
                comp = abs(height_feature[:, y] - height_feature[:, y-1]) <= threshold
                count[:, y] = np.where(comp, count[:, y-1] + 1, 0)
        else:
            count[:, -1] = 1
            for y in range(self.bin_size_y-2, -1, -1):
                if is_equal:
                    comp = abs(height_feature[:, y] - height_feature[:, y+1]) <= threshold
                else:
                    comp = height_feature[:, y] >= height_feature[:, y+1]
                count[:, y] = np.where(comp, count[:, y+1] + 1, 1)
        plane_feature = count

    return plane_feature


def print_plane_features(matrix):
    h = matrix
    features = [
        h,
        get_distance(h, x_ori=True),
        get_distance(h, x_ori=False),
        get_distance(h, x_ori=True, neg=True),
        get_distance(h, x_ori=False, neg=True),
        get_distance(h, x_ori=True, is_equal=False),
        get_distance(h, x_ori=False, is_equal=False)
    ]
    # Stack features: (x, y, 7)
    stacked = np.stack(features, axis=-1)
    
    for row in stacked:
        line = ' '.join(f"({','.join(str(int(v)) for v in cell)})" for cell in row)
        print(line)

# Beispielmatrix
container_matrix = np.array([
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [0,0,0,0,0,0,0,3,3]
], dtype=int)

# Ausgabe
print_plane_features(container_matrix)


TypeError: get_distance() missing 1 required positional argument: 'height_feature'

In [ ]:
import numpy as np

def get_distance_vec(height_feature, x_ori=True, is_equal=True, neg=False, threshold=0):
    h = height_feature
    if x_ori:
        diff = np.diff(h, axis=0)
    else:
        diff = np.diff(h, axis=1)
    
    if is_equal:
        comp = np.abs(diff) <= threshold
    else:
        comp = diff[:-1] >= 0 if x_ori else diff[:, :-1] >= 0
    
    # Für neg=True invertieren wir die Richtung
    if neg:
        comp = comp if x_ori else comp.T
        comp = np.vstack([np.zeros((1, comp.shape[1]), dtype=bool), comp])
        # Cumsum, reset auf 0 bei False
        count = np.zeros_like(h, dtype=int)
        for i in range(1, h.shape[0] if x_ori else h.shape[1]):
            count[i, :] = np.where(comp[i, :], count[i-1, :] + 1, 1)
        return count
    else:
        # Vorwärts-Variante
        comp = comp if x_ori else comp.T
        comp = np.vstack([comp, np.zeros((1, comp.shape[1]), dtype=bool)])
        count = np.ones_like(h, dtype=int)
        for i in range(h.shape[0]-2, -1, -1) if x_ori else range(h.shape[1]-2, -1, -1):
            if x_ori:
                count[i, :] = np.where(comp[i, :], count[i+1, :] + 1, 1)
            else:
                count[:, i] = np.where(comp[i, :], count[:, i+1] + 1, 1)
        return count

def print_plane_features_vec(matrix):
    h = matrix
    p_x = get_distance_vec(h, x_ori=True)
    p_y = get_distance_vec(h, x_ori=False)
    p_x_neg = get_distance_vec(h, x_ori=True, neg=True)
    p_y_neg = get_distance_vec(h, x_ori=False, neg=True)
    p_x_gap = get_distance_vec(h, x_ori=True, is_equal=False)
    p_y_gap = get_distance_vec(h, x_ori=False, is_equal=False)
    
    stacked = np.stack([h, p_x, p_y, p_x_neg, p_y_neg, p_x_gap, p_y_gap], axis=-1)
    
    for row in stacked:
        line = ' '.join(f"({','.join(str(int(v)) for v in cell)})" for cell in row)
        print(line)

# Testmatrix
container_matrix = np.array([
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [0,0,0,0,0,0,0,3,3]
], dtype=int)

print_plane_features_vec(container_matrix)


(5,3,4,1,1,9,9) (5,3,3,1,1,9,8) (5,3,2,1,2,9,7) (5,3,1,1,3,9,6) (4,3,5,1,0,9,5) (4,3,4,1,1,9,4) (4,3,3,1,2,9,3) (4,3,2,1,3,9,2) (4,3,1,1,4,9,1)
(5,2,4,1,1,8,9) (5,2,3,1,1,8,8) (5,2,2,1,2,8,7) (5,2,1,1,3,8,6) (4,2,5,1,0,8,5) (4,2,4,1,1,8,4) (4,2,3,1,2,8,3) (4,2,2,1,3,8,2) (4,2,1,1,4,8,1)
(5,1,4,2,1,7,9) (5,1,3,2,1,7,8) (5,1,2,2,2,7,7) (5,1,1,2,3,7,6) (4,1,5,2,0,7,5) (4,1,4,2,1,7,4) (4,1,3,2,2,7,3) (4,1,2,2,3,7,2) (4,1,1,2,4,7,1)
(2,5,6,0,1,6,7) (2,5,5,0,1,6,6) (2,5,4,0,2,6,5) (2,5,3,0,3,6,4) (2,5,2,0,4,6,3) (2,5,1,0,5,6,2) (0,6,1,0,0,6,1) (3,6,2,0,0,6,2) (3,6,1,0,1,6,1)
(2,4,6,1,1,5,7) (2,4,5,1,1,5,6) (2,4,4,1,2,5,5) (2,4,3,1,3,5,4) (2,4,2,1,4,5,3) (2,4,1,1,5,5,2) (0,5,1,1,0,5,1) (3,5,2,1,0,5,2) (3,5,1,1,1,5,1)
(2,3,6,2,1,4,7) (2,3,5,2,1,4,6) (2,3,4,2,2,4,5) (2,3,3,2,3,4,4) (2,3,2,2,4,4,3) (2,3,1,2,5,4,2) (0,4,1,2,0,4,1) (3,4,2,2,0,4,2) (3,4,1,2,1,4,1)
(2,2,6,3,1,3,7) (2,2,5,3,1,3,6) (2,2,4,3,2,3,5) (2,2,3,3,3,3,4) (2,2,2,3,4,3,3) (2,2,1,3,5,3,2) (0,3,1,3,0,3,1) (3,3,2,3,0,3,2) (3,3,1,3

# Neuer Versuch

In [6]:
import numpy as np

class Env():
    def __init__(self, bin_size_x, bin_size_y, bin_size_z, bin_size_ds_x, bin_size_ds_y, box_num, min_factor,
                 max_factor, feature_num=6, distance_threshold=0, gap_filling=False):

        self.max_step = 8192

        # Environment Parameters
        self.bin_size_x = bin_size_x
        self.bin_size_y = bin_size_y
        self.bin_size_z = bin_size_z
        self.bin_size_ds_x = bin_size_ds_x
        self.bin_size_ds_y = bin_size_ds_y
        self.box_num = box_num
        self.min_factor = min_factor
        self.max_factor = max_factor
        self.feature_num = feature_num + 1
        self.distance_threshold = distance_threshold
        self.gap_filling = gap_filling
        self.offline = True


        # Environment Variables
        self.gap = 0
        self.total_volume = 0
        self.max_index = None
        self.packing_result = []
        self.residual_box_num = box_num
        self.total_trunc_height = 0


        # Environment Constants
        self.rotation_matrix = np.array([[[1, 0, 0],
                                          [0, 1, 0],
                                          [0, 0, 1]],
                                         [[1, 0, 0],
                                          [0, 0, 1],
                                          [0, 1, 0]],
                                         [[0, 1, 0],
                                          [1, 0, 0],
                                          [0, 0, 1]],
                                         [[0, 0, 1],
                                          [1, 0, 0],
                                          [0, 1, 0]],
                                         [[0, 1, 0],
                                          [0, 0, 1],
                                          [1, 0, 0]],
                                         [[0, 0, 1],
                                          [0, 1, 0],
                                          [1, 0, 0]]])

        self.rotation_matrix_all = np.array([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
                                             [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
                                             [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0]])

        self.original_bin_height = np.zeros((self.bin_size_x, self.bin_size_y), dtype=np.float32)
        self.original_state_bin = self.get_bin_feature(self.original_bin_height)
        self.block_size_x = self.bin_size_x // self.bin_size_ds_x
        self.block_size_y = self.bin_size_y // self.bin_size_ds_y

        if self.bin_size_ds_y < self.bin_size_y or self.bin_size_ds_x < self.bin_size_x:
            self.original_state_bin_ds, self.original_max_index = self.down_sampling(self.original_state_bin)

    def get_distance(self, height_feature, x_ori, is_equal=True, neg=False, threshold=0):
        plane_feature = np.zeros_like(height_feature)

        if x_ori:
            if neg:
                count = np.zeros((1, self.bin_size_y))
                for x_index in range(1, self.bin_size_x):
                    comparison = abs(height_feature[x_index, :] - height_feature[x_index - 1, :]) <= threshold
                    count = np.where(comparison, count + 1, 0)
                    plane_feature[x_index, :] = count * 1
            else:
                count = np.ones((1, self.bin_size_y))
                for x_index in range(self.bin_size_x - 2, -1, -1):
                    if is_equal:
                        comparison = abs(height_feature[x_index, :] - height_feature[x_index + 1, :]) <= threshold
                    else:
                        comparison = height_feature[x_index, :] >= height_feature[x_index + 1, :]
                    count = np.where(comparison, count + 1, 1)
                    plane_feature[x_index, :] = count * 1
        else:
            if neg:
                count = np.zeros((1, self.bin_size_x))
                for y_index in range(1, self.bin_size_y):
                    comparison = abs(height_feature[:, y_index] - height_feature[:, y_index - 1]) <= threshold
                    count = np.where(comparison, count + 1, 0)
                    plane_feature[:, y_index] = count * 1
            else:
                count = np.ones((1, self.bin_size_x))
                for y_index in range(self.bin_size_y - 2, -1, -1):
                    if is_equal:
                        comparison = abs(height_feature[:, y_index] - height_feature[:, y_index + 1]) <= threshold
                    else:
                        comparison = height_feature[:, y_index] >= height_feature[:, y_index + 1]
                    count = np.where(comparison, count + 1, 1)
                    plane_feature[:, y_index] = count * 1

        return plane_feature

    def get_bin_feature(self, height_feature):
        h_feature = height_feature
        p_feature_x = self.get_distance(h_feature, x_ori=True)
        p_feature_y = self.get_distance(h_feature, x_ori=False)
        p_feature_x_neg = self.get_distance(h_feature, x_ori=True, neg=True)
        p_feature_y_neg = self.get_distance(h_feature, x_ori=False, neg=True)
        p_feature_x_gap = self.get_distance(h_feature, x_ori=True, is_equal=False)
        p_feature_y_gap = self.get_distance(h_feature, x_ori=False, is_equal=False)

        if self.feature_num == 5:
            bin_feature_state = np.stack(
                [h_feature, p_feature_x, p_feature_y, p_feature_x_neg, p_feature_y_neg],
                -1)
        else:
            bin_feature_state = np.stack(
                [h_feature, p_feature_x, p_feature_y, p_feature_x_neg, p_feature_y_neg, p_feature_x_gap,
                 p_feature_y_gap],
                -1)

        return bin_feature_state

    @staticmethod
    def generate_box_array(bin_size_x, bin_size_y, box_num, min_factor, max_factor):

        box_array_x = np.random.randint(int(bin_size_x * min_factor), int(bin_size_x * max_factor + 1), [box_num,])
        box_array_y = np.random.randint(int(bin_size_y * min_factor), int(bin_size_y * max_factor + 1), [box_num,])
        box_array_z = np.random.randint(int(min(bin_size_x,bin_size_y) * min_factor),
                                        int(max(bin_size_x,bin_size_y) * max_factor + 1),
                                        [box_num,])
        box_array = np.stack([box_array_x,box_array_y,box_array_z],-1)

        return box_array

    def down_sampling(self, bin_state):
        bin_size_x, bin_size_y, feature_num = bin_state.shape
        bin_state_split = np.stack(np.split(bin_state, self.bin_size_ds_x, 0), 0)
        bin_state_split = np.stack(np.split(bin_state_split, self.bin_size_ds_y, 2), 1).reshape(
            self.bin_size_ds_x * self.bin_size_ds_y, -1, feature_num)
        max_target = bin_state_split[:, :, 1] * bin_state_split[:, :, 2]
        max_idx = max_target.argmax(-1).reshape(-1, 1, 1)

        bin_state_ds = np.take_along_axis(bin_state_split, max_idx, 1).reshape(self.bin_size_ds_x, self.bin_size_ds_y,
                                                                               -1)

        return bin_state_ds, max_idx

    def down_sampling_mask(self, mask, max_index):
        box_num, ori_num, bin_size_x, bin_size_y = mask.shape
        mask_split = np.stack(np.split(mask, self.bin_size_ds_x, 2), 2)
        mask_split = np.stack(np.split(mask_split, self.bin_size_ds_y, 4), 3).reshape(box_num, ori_num,
                                                                                   self.bin_size_ds_x * self.bin_size_ds_y,
                                                                                   -1)
        mask_ds = np.take_along_axis(mask_split, max_index.reshape(1, 1, -1, 1), -1).reshape(box_num, ori_num,
                                                                                             self.bin_size_ds_x,
                                                                                             self.bin_size_ds_y)

        return mask_ds

    def reset(self, box_array=None):
        # reset parameter
        self.gap = 0
        self.total_volume = 0
        self.total_trunc_height = 0
        self.packing_result = []
        self.residual_box_num = self.box_num
        # generate state_box
        if box_array is None:
            box_array = self.generate_box_array(self.bin_size_x, self.bin_size_y, self.box_num, self.min_factor,
                                                self.max_factor)
        else:
            box_array = box_array * 1
        self.box_array = box_array * 1

        state_box = self.box_array

        # generate state_bin
        self.bin_height = self.original_bin_height * 1
        state_bin = self.original_state_bin * 1

        if self.bin_size_ds_y < self.bin_size_y or self.bin_size_ds_x < self.bin_size_x:  # need downsampling

            state_bin = self.original_state_bin_ds * 1
            self.max_index = self.original_max_index * 1
            packing_mask = self.get_mask_no_constraint(state_box, self.max_index)

        else:
            packing_mask = self.get_mask_no_constraint(state_box)

        self.state = (state_bin, state_box, packing_mask)

        return self.state

    def get_mask_no_constraint(self, state_box, max_index=None):
        x_residual_size = np.zeros((self.bin_size_ds_x, self.bin_size_ds_y)) + np.arange(self.bin_size_ds_x, 0, -1).reshape(-1,1) * self.block_size_x
        y_residual_size = np.zeros((self.bin_size_ds_x, self.bin_size_ds_y)) + np.arange(self.bin_size_ds_y, 0, -1) * self.block_size_y
        position_residual_size = np.stack([x_residual_size, y_residual_size], 2)

        available_box_num = self.residual_box_num

        box_array = state_box[:available_box_num] * 1  # residual_box_num x 3

        box_rotation_array = np.matmul(box_array, self.rotation_matrix_all).reshape(-1, 6,
                                                                                    3)  # residual_box_num x 6 x 3
        box_rotation_array = box_rotation_array[:, :, :2]

        if max_index is not None:
            max_index_size = np.stack([max_index / (self.block_size_y), max_index % (self.block_size_y)], -1)
            position_residual_size = position_residual_size - max_index_size.reshape(self.bin_size_ds_x,
                                                                                     self.bin_size_ds_y,
                                                                                     2)  # 10 x 10 x 2

        packing_available = box_rotation_array.reshape(-1, 6, 1, 1, 2) <= position_residual_size.reshape(1, 1,
                                                                                                         self.bin_size_ds_x,
                                                                                                         self.bin_size_ds_y,
                                                                                                         2)
        packing_available = packing_available.all(-1)

        packing_available = np.pad(packing_available, ((0, self.box_num - available_box_num), (0, 0), (0, 0), (0, 0)),
                                   constant_values=False)

        return ~packing_available

    def step(self, action: tuple, debug=True):
        a_i, a_xy, a_r = action

        if self.max_index is not None:
            sub_index = int(self.max_index[a_xy])
            a_x = a_xy // self.bin_size_ds_y * self.block_size_x + sub_index // self.block_size_y
            a_y = a_xy % self.bin_size_ds_y * self.block_size_y + sub_index % self.block_size_y
        else:
            a_x = a_xy // self.bin_size_y
            a_y = a_xy % self.bin_size_y

        box_shape = self.box_array[a_i]
        box_rotation_shape = np.matmul(box_shape, self.rotation_matrix_all).reshape(6, 3)
        box_shape_rot = box_rotation_shape[a_r]
        box_length, box_width, box_height = map(int, box_shape_rot)
        if debug:
            assert a_x + box_length <= self.bin_size_x and a_y + box_width <= self.bin_size_y
        place_pack = self.bin_height[a_x:a_x + box_length, a_y:a_y + box_width]
        a_z = np.max(place_pack)
        place_pack[:, :] = float(np.max(place_pack) + box_height)
        if self.bin_size_z < 10000 and debug:
            assert np.max(place_pack) <= self.bin_size_z
        self.packing_result.append([box_length, box_width, box_height, a_x, a_y, a_z])

        # generate state_box
        self.box_array = np.delete(self.box_array, a_i, 0)
        self.box_array = np.pad(
            self.box_array, ((0, self.box_num - self.box_array.shape[0]), (0, 0)), constant_values=-1e9)
        state_box = self.box_array

        self.residual_box_num -= 1

        # generate state_bin


        bin_height = self.bin_height
        state_bin = self.get_bin_feature(bin_height)
        if self.bin_size_ds_y < self.bin_size_y or self.bin_size_ds_x < self.bin_size_x:
            state_bin_ds, self.max_index = self.down_sampling(state_bin)

        if self.bin_size_ds_y < self.bin_size_y or self.bin_size_ds_x < self.bin_size_x:  # need downsampling
            if self.residual_box_num != 0:
                state_bin, self.max_index = self.down_sampling(state_bin)
                packing_mask = self.get_mask_no_constraint(state_box,self.max_index)
            else:
                state_bin, self.max_index = self.down_sampling(state_bin)
                packing_mask = np.ones((self.box_num, 6, self.bin_size_ds_x, self.bin_size_ds_y))
        else:
            if self.residual_box_num != 0:
                packing_mask = self.get_mask_no_constraint(state_box)
            else:
                packing_mask = np.ones((self.box_num, 6, self.bin_size_ds_x, self.bin_size_ds_y))

        self.packing_mask = packing_mask
        self.state = (state_bin, state_box, packing_mask)

        # reward
        self.total_volume += box_length * box_width * box_height
        total_bin_volume = self.bin_size_x * \
                           self.bin_size_y * float(np.max(self.bin_height))

        new_gap = total_bin_volume - self.total_volume
        reward = (self.gap - new_gap) / self.bin_size_x / self.bin_size_y
        self.gap = new_gap
        if self.bin_size_z < 10000:
            total_bin_volume = self.bin_size_x * self.bin_size_y * self.bin_size_z
        self.use_ratio = self.total_volume / total_bin_volume * 100

        # done
        if self.residual_box_num == 0 or packing_mask.all():
            done = True
            if self.bin_size_z < 10000:
                reward = reward - (self.bin_size_z - np.max(self.bin_height))
        else:
            done = False
        return self.state, reward, done




def test_env_with_matrix(container_matrix):
    bin_size_x, bin_size_y = container_matrix.shape
    bin_size_z = np.max(container_matrix) + 1  # nur für Test
    env = Env(
        bin_size_x=bin_size_x,
        bin_size_y=bin_size_y,
        bin_size_z=bin_size_z,
        bin_size_ds_x=bin_size_x,  # kein Downsampling
        bin_size_ds_y=bin_size_y,
        box_num=1,
        min_factor=1,
        max_factor=1,
        feature_num=6
    )

    bin_features = env.get_bin_feature(container_matrix)

    # Ausgabe formatieren: np.int64 -> int
    for x in range(bin_size_x):
        row_features = []
        for y in range(bin_size_y):
            row_features.append(tuple(int(f) for f in bin_features[x, y, :]))
        print(" ".join(map(str, row_features)))



container_matrix = np.array([
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [0,0,0,0,0,0,0,3,3]
], dtype=int)

test_env_with_matrix(container_matrix)



(5, 3, 4, 0, 0, 9, 9) (5, 3, 3, 0, 1, 9, 8) (5, 3, 2, 0, 2, 9, 7) (5, 3, 1, 0, 3, 9, 6) (4, 3, 5, 0, 0, 9, 5) (4, 3, 4, 0, 1, 9, 4) (4, 3, 3, 0, 2, 9, 3) (4, 3, 2, 0, 3, 9, 2) (4, 3, 0, 0, 4, 9, 0)
(5, 2, 4, 1, 0, 8, 9) (5, 2, 3, 1, 1, 8, 8) (5, 2, 2, 1, 2, 8, 7) (5, 2, 1, 1, 3, 8, 6) (4, 2, 5, 1, 0, 8, 5) (4, 2, 4, 1, 1, 8, 4) (4, 2, 3, 1, 2, 8, 3) (4, 2, 2, 1, 3, 8, 2) (4, 2, 0, 1, 4, 8, 0)
(5, 1, 4, 2, 0, 7, 9) (5, 1, 3, 2, 1, 7, 8) (5, 1, 2, 2, 2, 7, 7) (5, 1, 1, 2, 3, 7, 6) (4, 1, 5, 2, 0, 7, 5) (4, 1, 4, 2, 1, 7, 4) (4, 1, 3, 2, 2, 7, 3) (4, 1, 2, 2, 3, 7, 2) (4, 1, 0, 2, 4, 7, 0)
(2, 5, 6, 0, 0, 6, 7) (2, 5, 5, 0, 1, 6, 6) (2, 5, 4, 0, 2, 6, 5) (2, 5, 3, 0, 3, 6, 4) (2, 5, 2, 0, 4, 6, 3) (2, 5, 1, 0, 5, 6, 2) (0, 6, 1, 0, 0, 6, 1) (3, 6, 2, 0, 0, 6, 2) (3, 6, 0, 0, 1, 6, 0)
(2, 4, 6, 1, 0, 5, 7) (2, 4, 5, 1, 1, 5, 6) (2, 4, 4, 1, 2, 5, 5) (2, 4, 3, 1, 3, 5, 4) (2, 4, 2, 1, 4, 5, 3) (2, 4, 1, 1, 5, 5, 2) (0, 5, 1, 1, 0, 5, 1) (3, 5, 2, 1, 0, 5, 2) (3, 5, 0, 1, 1, 5, 0)
(2, 3, 6, 

In [ ]:
import numpy as np

# Deine Container-Matrix
container_matrix = np.array([
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [5,5,5,5,4,4,4,4,4],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [2,2,2,2,2,2,0,3,3],
    [0,0,0,0,0,0,0,3,3]
], dtype=int)

bin_size_x, bin_size_y = container_matrix.shape
threshold = 0  # wie in der Funktion

# plane_feature initialisieren
plane_feature = np.zeros_like(container_matrix)
count = np.zeros((1, bin_size_y))

# Schleife exakt wie in deinem Ausschnitt
for x_index in range(1, bin_size_x):
    comparison = abs(container_matrix[x_index, :] - container_matrix[x_index - 1, :]) <= threshold
    count = np.where(comparison, count + 1, 0)
    plane_feature[x_index, :] = count * 1

# Ausgabe
print("\nPlane Feature:\n", plane_feature)



Plane Feature:
 [[0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4 4]
 [0 0 0 0 0 0 5 5 5]]
